### Image Segmentation Project 1
###### The following program optimizes the ambrios functional using gradient descent method to use a gradieent descent method, the gradient of the functional needs

In [2]:
using Images, FileIO
using TestImages

In [3]:
# set the image to segment.
img = testimage("lighthouse");
m = size(img)[1];
n = size(img)[2];

In [5]:
# the following is an implementation of the potential function
# along with the function z wtih z being from 0 and 1

epsilon = 0.02;
beta = 0.02;
alpha = 0.02;

function setUp_grad_z(m,n)
    function grad_z(x)
        return (2 / (2*m+n))^2 + (1 / (2*m+n)^2)
    end;
    return grad_z
end;

function setUp_z(m,n)
    function z(x)
        return (2*x[1] + x[2])/(2*m+n);
    end;
    return z;
end;

function phi(z)
    
    return z^4 - 2*z^3 + z^2
end;
grad_z = setUp_grad_z(m,n);
z = setUp_z(m,n);
phi_squared = x -> phi(z(x));